Модель предсказывает автора по куску текста

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import collections
from sklearn.cross_validation import train_test_split


data = pd.read_csv('dataset.csv')
data.shape

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(19579, 3)

In [2]:
X = data.drop(('author'), axis=1)
y = data['author']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)


In [3]:
sentence = [i for i in X_train['text']]
authors = [i for i in y_train]

info = {sentence[i]:authors[i] for i in range(len(X_train['text']))}


In [7]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

text_train = X_train['text'].values.tolist()
author_train = y_train.values.tolist()

corp = text_train + author_train

nltk_tok = [nltk.word_tokenize(sent) for sent in corp]


[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
model = Word2Vec(nltk_tok, min_count=1, size=40, hs=1, negative=0)

In [9]:
# Преобразвание обучающих данных

new_text_vec = []
for i in X_train['text']:
    text = model.score([i])
    new_text_vec.append(text)

In [10]:
model.score(['Our conversation was now long, earnest, uninterrupted, and totally unreserved.'.split()])

array([-42.939392], dtype=float32)

In [11]:
text = []
for i in range(len(new_text_vec)):
    text.append(float(new_text_vec[i].view()[0]))
X_train_new = pd.DataFrame(columns=['id','text'])
X_train_new['text'] = text
X_train_new['id'] = [i for i in range(len(text))]
X_train_new['author'] = y_train
X_train_new = X_train_new.fillna(X_train_new.mean())

In [12]:
X_train_new['author'] = X_train_new['author'].fillna(0)
X_train_new = X_train_new[X_train_new.author != float(0)]


In [13]:
# Преобразование тестовых данных

text_vec = []
for i in X_test['text']:
    text = model.score([i])
    text_vec.append(text)

In [14]:
text_test = []
for i in range(len(text_vec)):
    text_test.append(float(text_vec[i].view()[0]))
X_test_new = pd.DataFrame(columns=['id','text'])
X_test_new['text'] = text_test
X_test_new['id'] = [i for i in range(len(text_test))]
X_test_new['author'] = y_test
X_test_new = X_test_new.fillna(X_test_new.mean())

In [15]:
X_test_new = X_test_new.fillna(0)
X_test_new = X_test_new[X_test_new.author != 0]

In [16]:
X_train = X_train_new.drop(('author'), axis=1)
y_train = X_train_new['author']
X_test = X_test_new.drop(('author'), axis=1)
y_test = X_test_new['author']


In [17]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=45, max_depth = 24, min_samples_leaf=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [18]:
from sklearn.metrics import f1_score, confusion_matrix

f1_score(y_test, y_pred, average='macro')
# confusion_matrix(y_test, y_pred)

0.3101109450235746

In [19]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
metrics.accuracy_score(y_test, y_pred)


0.3832301631963984